In [3]:
import numpy as np
import pandas as pd
import cv2 as cv
import pytesseract as ts

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

## Import an Image

In [14]:
imgPaths = glob('/opt/Python/pyOCR/images/*.jpg')

In [15]:
imgPath = imgPaths[0]
_, filename = os.path.split(imgPath)

card_29.jpg


## Extract the Text

In [20]:
image = cv.imread(imgPath)
data = ts.image_to_data(image)

dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:], columns=dataList[0])

In [22]:
df.dropna(inplace=True)
df['conf'] = df['conf'].astype(int)
textData = df.query('conf >= 30')

businessCard = pd.DataFrame()
businessCard['text'] = textData['text']
businessCard['id'] = filename

## Loop over all Images

In [26]:
allBusinessCards = pd.DataFrame(columns=['id', 'text'])

for imgPath in tqdm(imgPaths,desc="Business Card"):

    # Get Filenames
    _, filename = os.path.split(imgPath)
    # Extract Data
    image = cv.imread(imgPath)
    data = ts.image_to_data(image)
    # Write Data to Frame
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:], columns=dataList[0])
    # Drop Everything that is not useful
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(int)
    textData = df.query('conf >= 30')
    # Define a Business Card Entity
    businessCard = pd.DataFrame()
    businessCard['text'] = textData['text']
    businessCard['id'] = filename
    # Add Card to All Cards
    allBusinessCards = pd.concat((allBusinessCards,businessCard))

Business Card: 100%|███████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:16<00:00,  3.26it/s]


In [28]:
allBusinessCards.to_csv('businessCards.csv', index=False)